In [138]:
import os
import pandas as pd
import numpy as np

In [139]:
dfs = []
for filename in os.listdir("Golf Club"):
    dfs.append(pd.read_excel("Golf Club/"+ filename))
    if len(dfs) >= 4:
        break

In [140]:
#check for stationarity
from statsmodels.tsa.stattools import adfuller
"""""
for df in dfs:
    for col in ['Volts_avg', 'Volts_std', 'VoltsA_avg', 'VoltsA_std', 'Amps_avg', 'Amps_std',
    'AmpsA_avg', 'AmpsA_std', 'ReactivePower_avg', 'ReactivePower_std', 'ApparentPower_avg', 'ApparentPower_std',
       'Frequency_avg', 'Frequency_std', 'Humidity_avg', 'Temp_avg', 'RealPower_avg']:
        temp = (adfuller(df[col].to_numpy()))
        if (temp[0] > 0):
            print(col)
"""""


# all columns above are stationary (ADFuller < 0) (I'm very confused as to why temperature is stationary? perhaps because of daily variance?)

'""\nfor df in dfs:\n    for col in [\'Volts_avg\', \'Volts_std\', \'VoltsA_avg\', \'VoltsA_std\', \'Amps_avg\', \'Amps_std\',\n    \'AmpsA_avg\', \'AmpsA_std\', \'ReactivePower_avg\', \'ReactivePower_std\', \'ApparentPower_avg\', \'ApparentPower_std\',\n       \'Frequency_avg\', \'Frequency_std\', \'Humidity_avg\', \'Temp_avg\', \'RealPower_avg\']:\n        temp = (adfuller(df[col].to_numpy()))\n        if (temp[0] > 0):\n            print(col)\n'

In [141]:
# standardize values
for df in dfs:
    df.drop(columns=['Timecode', 'Datetime'], inplace=True)
    for col in ['Volts_avg', 'Volts_std', 'VoltsA_avg', 'VoltsA_std', 'Amps_avg', 'Amps_std',
    'AmpsA_avg', 'AmpsA_std', 'ReactivePower_avg', 'ReactivePower_std', 'ApparentPower_avg', 'ApparentPower_std',
       'Frequency_avg', 'Frequency_std', 'Humidity_avg', 'Temp_avg', 'RealPower_avg']:
        df[col] = ((df[col] - df[col].mean()) / df[col].std())

In [142]:
import math
# make time features sinuosoidal
for df in dfs:
    df['Month'] = df['Month'].astype(float)
    df['Day'] = df['Day'].astype(float)
    df['Hour'] = df['Hour'].astype(float)
    df['Minute'] = df['Minute'].astype(float)
    df['Weekday'] = df['Weekday'].astype(float)

    for i, row in df.iterrows():
        tempMonth = df.at[i, 'Month']
        df.at[i, 'Month'] = math.sin((2 * math.pi * tempMonth) / 12)
        tempDay = df.at[i, 'Day']
        if int(tempMonth) in [1, 3, 5, 7, 8, 10, 12]:
            df.at[i, 'Day'] = math.sin((2 * math.pi * tempDay) / 31)
        elif int(tempMonth) in [4, 6, 9, 11]:
            df.at[i, 'Day'] = math.sin((2 * math.pi * tempDay) / 30)
        elif (int(tempMonth) in [2]) and (int(df.at[i, 'Year']) % 4 == 0):
            df.at[i, 'Day'] = math.sin((2 * math.pi * tempDay) / 29)
        elif int(tempMonth) in [2] and int(df.at[i, 'Year']) % 4 != 0:
            df.at[i, 'Day'] = math.sin((2 * math.pi * tempDay) / 29)
        else:
            print("Issue in data")
        tempHour = df.at[i, 'Hour']
        df.at[i, 'Hour'] = math.sin((2 * math.pi * tempHour) / 23)
        tempMinute = df.at[i, 'Minute']
        df.at[i, 'Minute'] = math.sin((2 * math.pi * tempMinute) / 60)
        tempWeekday = df.at[i, 'Weekday']
        df.at[i, 'Weekday'] = math.sin((2 * math.pi * tempWeekday) / 7)